# [Day 21](https://adventofcode.com/2020/day/21): Allergen Assessment

In [1]:
import re

with open("../data/21.txt", "r") as f:
    foods = re.findall(r"(.*) \(contains (.*)\)", f.read())
    foods = [(ing.split(), agn.split(", ")) for ing, agn in foods]
    ings, agns = zip(*foods)

## Part 1

In [2]:
import numpy as np
from itertools import chain

def danger(agns, ings):
    hasagn, _   = subsets(agns)
    hasing, ing = subsets(ings)
    dangerous = choice(maybedangerous(hasagn, hasing))
    return hasing, ing, dangerous

def subsets(xss):
    xs = sorted(set(chain.from_iterable(xss)))
    enum = {x: i for i, x in enumerate(xs)}
    ss = np.zeros((len(xss), len(xs)), dtype=int)
    for i, xs_ in enumerate(xss):
        ss[i, [enum[x] for x in xs_]] = 1
    return ss, np.array(xs)

def maybedangerous(hasagn, hasing):
    cnt = hasagn.T @ hasing
    return cnt == cnt.max(axis=1, keepdims=True)

# cf. Day 16
def choice(p):
    """Find a choice for a boolean matrix."""
    ch, a = np.empty((len(p), 2), dtype=int), 0
    indices = np.einsum("cab->abc", np.indices(p.shape))
    while len(p):
        unamb, amb = unambiguous(p)
        ch[a:(a := a + unamb.sum())] = indices[unamb]
        p, indices = p[amb], indices[amb]
    return ch[ch[:, 0].argsort(), 1]

def unambiguous(p):
    """Unambiguous choices for a boolean matrix."""
    a = p.sum(axis=1) == 1
    b = p[a].sum(axis=0) == 1
    unamb = np.zeros(p.shape, dtype=bool)
    unamb[ab] = p[(ab := np.ix_(a, b))]
    amb = np.ix_(~a | ~p[:, b].any(axis=1), ~b)
    return unamb, amb  

hasing, ing, dangerous = danger(agns, ings)
assert 1882 == hasing.sum() - hasing[:, dangerous].sum()

## Part 2

In [3]:
canonicaldangerous = ",".join(ing[dangerous])
assert "xgtj,ztdctgq,bdnrnx,cdvjp,jdggtft,mdbq,rmd,lgllb" == canonicaldangerous